In [1]:
using DrWatson

In [2]:
@quickactivate "RxInferThesisExperiments"

In [14]:
using RxInferThesisExperiments, ForneyLab, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, ProgressMeter, Dates

In [4]:
const bfolder = datadir("hgf", "forneylab")

"/Users/bvdmitri/.julia/dev/thesis/data/hgf/forneylab"

In [5]:
# Pregenerate paths for benchmark data
mkpath(bfolder);

In [6]:
const environment = HGFEnvironment()

HGFEnvironment()

In [7]:
# Include the model specification
include(srcdir("models", "forneylab", "hgf.jl"));

In [8]:
function run_benchmark(params)
    @unpack T, niterations, seed = params
    
    zstates, xstates, observations = rand(StableRNG(seed), environment, T);
    model    = hgf(environment.kappa, environment.omega)
    result   = run_inference(model, observations; iterations = niterations)
    e_states = extract_posteriors(T, result)
    z_amse   = compute_amse(zstates, e_states[:z])
    x_amse   = compute_amse(xstates, e_states[:x])
    
    benchmark_modelcreation = @benchmark hgf(environment.kappa, environment.omega; force = true)
    
    benchmark_inference = @benchmark run_inference(model, observations; iterations = $niterations) setup=begin
        model = hgf($(environment.kappa), $(environment.omega))
        zstates, xstates, observations = rand(StableRNG($seed), environment, $T);
    end
    
    emse = compute_emse(seed) do _seed
        local zstates, xstates, observations = rand(StableRNG(_seed), environment, T);
        local model    = hgf(environment.kappa, environment.omega)
        local result   = run_inference(model, observations; iterations = niterations)
        local e_states = extract_posteriors(T, result)
        return [ compute_amse(zstates, e_states[:z]), compute_amse(xstates, e_states[:x]) ]
    end
    
    z_emse = emse[1]
    x_emse = emse[2]
    
    states = (z = zstates, x = xstates)
    
    output = @strdict T niterations seed states e_states observations z_amse x_amse z_emse x_emse benchmark_modelcreation benchmark_inference
    
    return output
end

run_benchmark (generic function with 1 method)

In [21]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"           => [ 10, 20, 30, 100 ],
    "niterations" => [ 3, 10 ],
    "seed"        => [ 42 ]
));

In [22]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    result, _ = produce_or_load(run_benchmark, bfolder, params; tag = false, force = false)
    return result
end;

[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/hgf/forneylab/T=100_niterations=3_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/hgf/forneylab/T=100_niterations=3_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/hgf/forneylab/T=10_niterations=10_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/hgf/forneylab/T=10_niterations=10_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/hgf/forneylab/T=20_niterations=10_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/hgf/forneylab/T=20_niterations=10_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/hgf/forneylab/T=30_niterations=10_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/hgf/forneylab/T=30_niterations=10_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/

In [23]:
sort(prepare_benchmarks_table(bfolder), [ :T ])

[ Info: Scanning folder /Users/bvdmitri/.julia/dev/thesis/data/hgf/forneylab for result files.
[ Info: Added 8 entries.


Row,T,z_emse,x_emse,z_amse,niterations,seed,x_amse,inference,creation
,Int64?,Float64?,Float64?,Float64?,Int64?,Int64?,Float64?,Tuple…?,Tuple…?
1,10,1.14916,0.1582,1.03239,10,42,0.224379,"(4.42955e7, 5.2719e7, 0.0)","(20.5719, 24.2844, 0.0)"
2,10,1.32345,0.158711,1.18141,3,42,0.231314,"(1.26549e7, 1.39382e7, 0.0)","(20.0281, 22.458, 0.0)"
3,20,0.767198,0.154245,0.586204,10,42,0.187199,"(9.22362e7, 1.03299e8, 3.5976e6)","(21.8494, 28.0149, 0.0)"
4,20,0.881845,0.155043,0.663982,3,42,0.190732,"(2.57316e7, 3.00252e7, 0.0)","(20.8613, 23.7992, 0.0)"
5,30,0.59447,0.152034,0.432486,10,42,0.165137,"(1.39433e8, 1.49582e8, 3.5124e6)","(21.2, 25.1332, 0.0)"
6,30,0.681506,0.152781,0.488609,3,42,0.167435,"(3.85109e7, 4.28757e7, 0.0)","(20.1266, 22.6136, 0.0)"
7,100,0.240489,0.154593,0.161371,10,42,0.153471,"(4.85097e8, 5.24958e8, 2.30265e7)","(20.4794, 24.596, 0.0)"
8,100,0.274408,0.155031,0.18027,3,42,0.154218,"(1.42926e8, 1.60292e8, 3.95558e6)","(20.6894, 23.634, 0.0)"


# Versions

In [12]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin22.4.0)
  CPU: 12 × Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, skylake)
  Threads: 2 on 12 virtual cores


In [13]:
] status

Project RxInferThesisExperiments v1.0.0
Status `~/.julia/dev/thesis/Project.toml`
  [b5ca4192] AdvancedVI v0.2.3
  [6e4b80f9] BenchmarkTools v1.3.2
  [76274a88] Bijectors v0.12.4
  [35d6a980] ColorSchemes v3.21.0
  [a93c6f00] DataFrames v1.5.0
  [31c24e10] Distributions v0.25.95
  [634d3b9d] DrWatson v2.12.5
  [442a2c76] FastGaussQuadrature v0.5.1
  [9fc3f58a] ForneyLab v0.12.0
  [f6369f11] ForwardDiff v0.10.35
  [14197337] GenericLinearAlgebra v0.3.11
  [19dc6840] HCubature v1.5.1
  [7073ff75] IJulia v1.24.0
  [b964fa9f] LaTeXStrings v1.3.0
  [bdcacae8] LoopVectorization v0.12.159
  [3bd65402] Optimisers v0.2.18
  [8314cec4] PGFPlotsX v1.6.0
  [e4faabce] PProf v2.2.2
  [91a5bcdd] Plots v1.38.15
  [92933f4c] ProgressMeter v1.7.2
  [37e2e3b7] ReverseDiff v1.14.6
  [86711068] RxInfer v2.11.0
  [860ef19b] StableRNGs v1.0.0
  [aedffcd0] Static v0.8.7
  [90137ffa] StaticArrays v1.5.25
⌅ [2913bbd2] StatsBase v0.33.21
  [4c63d2b9] StatsFuns v1.3.0
  [fce5fe82] Turing v0.25.1
  [37e2e46d] Line